In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import pyreadr

In [42]:
with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/train_groups.pkl', 'rb') as f:
    train_groups = pickle.load(f)

with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/train_y.pkl', 'rb') as f:
    train_y = pickle.load(f)

with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/test_groups.pkl', 'rb') as f:
    test_groups = pickle.load(f)

with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/train_X.pkl', 'rb') as f:
    train_X = pickle.load(f)

    
with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_training/data/model_df.pkl', 'rb') as f:
    model_df = pickle.load(f)
with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_training/data/season_context.pkl', 'rb') as f:
    season_context = pickle.load(f)


# model_df = pyreadr.read_r('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/data_ingestion/data/modeling_df.rds')[None]

FullData = pyreadr.read_r('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/data_ingestion/data/FullData.rds')[None]

In [38]:
FullData = pyreadr.read_r('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/data_ingestion/data/FullData.rds')[None]


with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)


with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/full_predictions.pkl', 'rb') as f:
    full_predictions = pickle.load(f)

In [49]:
import mlflow
from mlflow import MlflowClient

client = MlflowClient()
mlflow.set_tracking_uri('http://localhost:5000')
# Set model version alias
model_name = "playoff_prediction_model"
# model_version_alias = "legendary-cub-637"
model_version = 7

model = mlflow.sklearn.load_model(f"models:/{model_name}/{model_version}")


In [50]:

output_predictions = model.predict(full_predictions.drop(columns = 'season'))
output_predictions = pd.Series(output_predictions, index=full_predictions.index)

updated_prediction_set = model_df.copy()
updated_prediction_set['predicted_label'] = output_predictions
updated_prediction_set = updated_prediction_set[['predicted_label'] + [col for col in updated_prediction_set.columns if col != 'predicted_label']]


final_predicitons = pd.merge(FullData, updated_prediction_set, how = 'inner', on = ['WinsPR', 'season', 'OTier', 'DTier', 'OrollTier', 'DrollTier'])
final_predicitons = final_predicitons[['predicted_label', 'label'] + [col for col in final_predicitons.columns if col not in ['predicted_label', 'label']]]


# Regression check

In [13]:
FullData = pyreadr.read_r('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/data_ingestion/data/FullData.rds')[None]


with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/model_df.pkl', 'rb') as f:
    model_df = pickle.load(f)


with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/full_predictions.pkl', 'rb') as f:
    full_predictions = pickle.load(f)

In [14]:
import mlflow
from mlflow import MlflowClient

client = MlflowClient()
mlflow.set_tracking_uri('http://localhost:5000')
# Set model version alias
model_name = "playoff_prediction_model_regression"
# model_version_alias = "legendary-cub-637"
model_version = 4

model = mlflow.sklearn.load_model(f"models:/{model_name}/{model_version}")


In [15]:

output_predictions = model.predict(full_predictions.drop(columns = 'season'))
output_predictions = pd.Series(output_predictions, index=full_predictions.index)

updated_prediction_set = model_df.copy()
updated_prediction_set['predicted_label'] = output_predictions
updated_prediction_set = updated_prediction_set[['predicted_label'] + [col for col in updated_prediction_set.columns if col != 'predicted_label']]


final_predicitons = pd.merge(FullData, updated_prediction_set, how = 'inner', on = ['WinsPR', 'season', 'OTier', 'DTier', 'OrollTier', 'DrollTier'])
final_predicitons = final_predicitons[['predicted_label', 'label'] + [col for col in final_predicitons.columns if col not in ['predicted_label', 'label']]]


In [17]:
import xgboost as xgb
model.save_model('production_model.model')

c:\Users\sethl\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [22:11:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


# Compare to old model

In [6]:
import xgboost as xgb
import pickle
import sklearn.metrics as metrics

# Load the model from the file
bst = xgb.Booster()
bst.load_model("C:/Users/sethl/OneDrive/R/R files/NFL/Shiny Apps/- Shiny App Updates/Update Files/season_prediction_regression.model")

def evaluate_regression_model(model, X, y, sw=None, type='test'):
    """Evaluate a regression model using various metrics."""
    # Make predictions on the provided set
    predictions = model.predict(X)
    
    # Calculate evaluation metrics
    mse = metrics.mean_squared_error(y, predictions, sample_weight=sw)
    rmse = np.sqrt(metrics.mean_squared_error(y, predictions, sample_weight=sw))
    mae = metrics.mean_absolute_error(y, predictions, sample_weight=sw)
    r2 = metrics.r2_score(y, predictions, sample_weight=sw)
    
    print(f"{type} MSE: {mse:.2f}")
    print(f"{type} RMSE: {rmse:.2f}")
    print(f"{type} MAE: {mae:.2f}")
    print(f"{type} R²: {r2:.2f}")

    results = {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

    return predictions, results


with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/test_X.pkl', 'rb') as f:
    test_X = pickle.load(f)
with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)
with open('C:/Users/sethl/OneDrive/Programming Stuff/Modeling Folder/NFL Models/playoff_predictor/model_evaluation/data/test_sw.pkl', 'rb') as f:
    test_sw = pickle.load(f)

dtest = xgb.DMatrix(test_X)

test_predictions, results = evaluate_regression_model(bst, dtest, test_y, test_sw, type='test')

XGBoostError: [15:01:25] C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:1483: Check failed: learner_model_param_.num_feature >= p_fmat->Info().num_col_ (17 vs. 20) : Number of columns does not match number of features in booster.